In [1]:
import pandas as pd
import os

folderpath_LinerAir = r"C:\Users\Ployphet.veerasak\Desktop\Defect\Liner air\Defect actual\RawData" #os.path.join(os.getcwd(), "RawData")
file_LinerAir =  [fLA for fLA in os.listdir(folderpath_LinerAir) if fLA.endswith(".xlsx")]

dfsLinerAir = []
for LA in file_LinerAir:
    filepath_LinerAir = os.path.join(folderpath_LinerAir, LA)
    dfLinerAir = pd.read_excel(filepath_LinerAir)
    dfsLinerAir.append(dfLinerAir)

LinerAirCombined = pd.concat(dfsLinerAir, ignore_index = True)
LinerAirFiltered = LinerAirCombined[LinerAirCombined["Defect name"] == "LINER AIR"]
LinerAirFiltered = LinerAirFiltered.sort_values(by=["Registered Date"], ascending = True)
LinerAirFiltered["Registered Date"] = pd.to_datetime(LinerAirFiltered["Registered Date"])

def adjust_date(row):
    registered_time = row["Registered Date"].time()
    if registered_time < pd.to_datetime("06:00:00").time():
        return (row["Registered Date"] - pd.Timedelta(days=1)).date()
    else:
        return row["Registered Date"].date()

LinerAirFiltered["Date"] = LinerAirFiltered.apply(adjust_date, axis=1)
LinerAirFiltered = LinerAirFiltered.dropna(how = "all", inplace = False)

#-----------------------------------------------------------------------------------------------------------------------

summary_defect_lot_path = r"C:\Users\Ployphet.veerasak\Desktop\Defect\Liner air\Defect Lot\Summary Defect Lot.xlsx"

# Read file without headers and set the first row as headers
dfDefectLotCombined = pd.read_excel(summary_defect_lot_path, header=None)
dfDefectLotCombined.columns = dfDefectLotCombined.iloc[0]
dfDefectLotCombined = dfDefectLotCombined[1:].reset_index(drop=True)

# Keep relevant columns and remove duplicates
dfDefectLotCombined = dfDefectLotCombined[["Barcode", "IL cart", "Side cart"]]
dfDefectLotCombined = dfDefectLotCombined.drop_duplicates(subset="Barcode")

# Merge with LinerAirFiltered
LinerAirFiltered = pd.merge(LinerAirFiltered, dfDefectLotCombined, on="Barcode", how="left")

#-----------------------------------------------------------------------------------------------------------------------

LinerAirScrap = LinerAirFiltered[LinerAirFiltered["QA Rank"] == "Scrap"]
LinerAirRework = LinerAirFiltered[LinerAirFiltered["QA Rank"] == "Rework"]

LinerAirFiltered.to_csv("LinerAirRaw.csv", index = False)
LinerAirScrap.to_csv("LinerAirScrap.csv", index = False)
LinerAirRework.to_csv("LinerAirRework.csv", index = False)

pivotScrap_table = pd.pivot_table(
    LinerAirScrap,
    index="Date",
    columns="Production Code",
    values="Defect name",
    aggfunc="count",
    fill_value=0
)
pivotScrap_reset = pivotScrap_table.reset_index()
meltedScrap_df = pd.melt(
    pivotScrap_reset,
    id_vars=["Date"],
    var_name="Size",
    value_name="Defect Count"
)
meltedScrap_df.to_csv("LinerAirScrap_Pivot.csv", index=False)

pivotRework_table = pd.pivot_table(
    LinerAirRework,
    index="Date",
    columns="Production Code",
    values="Defect name",
    aggfunc="count",
    fill_value=0
)
pivotRework_reset = pivotRework_table.reset_index()
meltedRework_df = pd.melt(
    pivotRework_reset,
    id_vars=["Date"],
    var_name="Size",
    value_name="Defect Count"
)
meltedRework_df.to_csv("LinerAirRework_Pivot.csv", index=False)